In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('../data/data.csv')
StarPlayer = pd.read_csv('../data/StarPlayer.csv')
StarPres = pd.read_csv('../data/StarPresence.csv')

In [6]:
# create numstars
numstars = (StarPlayer.groupby(['Year','Team'])
                  .size()
                  .reset_index(name='NumStars')\
                    .fillna({'NumStars': 0}))
numstars.head()

,Year,Team,NumStars
0,2011,Atlanta Hawks,1
1,2011,Chicago Bulls,1
2,2011,Dallas Mavericks,1
3,2011,Los Angeles Lakers,2
4,2011,Menphis Grizzlies,1


In [ ]:
# merge star player and star presence
star = (df
      .merge(StarPres, on=['Year','Team'], how='left')
      .merge(numstars, on=['Year','Team'], how='left'))

# fillna and convert to int
star[['StarPresence','NumStars']] = star[['StarPresence','NumStars']].fillna(0).astype(int)

In [10]:
# add star arrival
star = star.sort_values(['Team','Year'])
star['StarArrival'] = (
    star.groupby('Team')['StarPresence']
      .diff()               # ΔPresence
      .fillna(0)
      .eq(1)                # True if 0→1
      .astype(int)
)
star.to_csv('../data/data_star.csv', index=False)